In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [54]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
transform = transforms.Compose([
                    transforms.Resize((32,32)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

root = "/content/drive/My Drive/Colab Notebooks/CS221/dataset"
full_set = datasets.ImageFolder(root=root, transform=transform)
train_size = int(0.8 * len(full_set))
test_size = len(full_set) - train_size
train_set, test_set = torch.utils.data.random_split(full_set, [train_size, test_size])


train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, num_workers=1)


In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()      
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

net = Net()

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [58]:
for epoch in range(1):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
    
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')


[1,    20] loss: 0.705
[1,    40] loss: 0.697
[1,    60] loss: 0.703
[1,    80] loss: 0.687
[1,   100] loss: 0.699
[1,   120] loss: 0.690
[1,   140] loss: 0.691
[1,   160] loss: 0.691
[1,   180] loss: 0.697
[1,   200] loss: 0.702
[1,   220] loss: 0.683
[1,   240] loss: 0.681
[1,   260] loss: 0.668
[1,   280] loss: 0.697
[1,   300] loss: 0.678
[1,   320] loss: 0.686
[1,   340] loss: 0.718
[1,   360] loss: 0.680
[1,   380] loss: 0.697
[1,   400] loss: 0.683
[1,   420] loss: 0.681
[1,   440] loss: 0.669
[1,   460] loss: 0.659
[1,   480] loss: 0.701
[1,   500] loss: 0.635
[1,   520] loss: 0.673
[1,   540] loss: 0.606
[1,   560] loss: 0.732
[1,   580] loss: 0.693
[1,   600] loss: 0.672
[1,   620] loss: 0.640
[1,   640] loss: 0.648
[1,   660] loss: 0.662
[1,   680] loss: 0.747
[1,   700] loss: 0.713
[1,   720] loss: 0.658
[1,   740] loss: 0.620


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


[1,   760] loss: 0.700
[1,   780] loss: 0.592
[1,   800] loss: 0.708
[1,   820] loss: 0.695
[1,   840] loss: 0.603
[1,   860] loss: 0.619
[1,   880] loss: 0.657
[1,   900] loss: 0.727
[1,   920] loss: 0.691
[1,   940] loss: 0.664
[1,   960] loss: 0.542
[1,   980] loss: 0.520
[1,  1000] loss: 0.593
[1,  1020] loss: 0.559
[1,  1040] loss: 0.585
[1,  1060] loss: 0.633
[1,  1080] loss: 0.628
[1,  1100] loss: 0.486
[1,  1120] loss: 0.616
[1,  1140] loss: 0.518
[1,  1160] loss: 0.722
[1,  1180] loss: 0.620
[1,  1200] loss: 0.611
[1,  1220] loss: 0.707
[1,  1240] loss: 0.488
[1,  1260] loss: 0.728
[1,  1280] loss: 0.724
Finished Training


In [59]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 6. Skipping tag 41486
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag))


Accuracy of the network on the 10000 test images: 65 %
